In [1]:
%run talktools.py

## Lecture 00: Advanced Language Concepts

We're using Python 3.5.X...You should be at least at IPython v5.0.0 for this. 

If not:
<code bash>
conda update conda 
conda update ipython juypter
</code>

Then fire up the notebook
<code>
$ cd DataFiles_and_Notebooks/00_AdvancedPythonConcepts/
$ jupyter notebook
</code>

**Mr. Robot uses the CLI, but we'll use the Jupyter notebook.**

<img src="http://i.imgur.com/4Qwsxx6.png" width="60%">

https://www.reddit.com/r/Python/comments/3uxclx/noticed_some_python_code_in_the_tv_show_mr_robot/

<center>
<p class="gap03"</p>
<p class="talk_title">Useful Advanced Concepts</p>
<p class="gap03"</p>
</center>
<ul>
 <li> ``OrderedDict``, ``namedtuple``
 <li> building iterators (Classes & generating functions)
 <li> ``with`` statements (Context managers)
 <li> decorators
</ul>

## OrderedDict ##

not a core type like ``set``, ``dict``, ``list``, ``tuple``...
but still very useful.

In [2]:
a = {"cal": "wow", "stanford": "meh"}
b = {"stanford": "meh", "cal": "wow"}

In [3]:
a == b

True

In [4]:
for k in a.keys(): print(k,)
print("\n" + "*"*10)
for v in a.values(): print(v,)
print("\n" + "*"*10)
for k, v in a.items(): print(k, v)

cal
stanford

**********
wow
meh

**********
cal wow
stanford meh


In [5]:
from collections import OrderedDict

In [6]:
c = OrderedDict()

In [7]:
type(c)

collections.OrderedDict

In [8]:
c.update({"best schools": "cal"})
c.update({"worst schools": "you know"})
d = OrderedDict({"worst schools": "you know"})
d.update({"best schools": "cal"})

In [9]:
c == d

False

In [10]:
print(c)
print(d)

OrderedDict([('best schools', 'cal'), ('worst schools', 'you know')])
OrderedDict([('worst schools', 'you know'), ('best schools', 'cal')])


In [11]:
{1: "a", "n":"cat"}

{1: 'a', 'n': 'cat'}

In [12]:
c.popitem()

('worst schools', 'you know')

In [13]:
d.popitem()

('best schools', 'cal')

In [14]:
print("c=", c)
print("d=", d)

c= OrderedDict([('best schools', 'cal')])
d= OrderedDict([('worst schools', 'you know')])


 &nbsp;

`OrderedDict` has all the same methods as `dict` types but includes the `.popitem()` method (which sort of like doing a `.pop()` on a list without any arguements.

In [15]:
d = OrderedDict()
d.update({"best schools": "cal", 
          "worst schools": "you know"})
e = OrderedDict({"worst schools": "you know",
                 "best schools": "cal"})

In [16]:
d == e

True

"The `OrderedDict` constructor and `update()` method both accept keyword arguments, but their order is lost because Python’s function call semantics pass-in keyword arguments using a regular unordered dictionary."

https://docs.python.org/3/library/collections.html#collections.OrderedDict

In [17]:
# other ways to update OrderedDict
d.update(cal="I mean wow", stanford="what's French for 'meh'?")
d.update([("Famous Trumps",["Donald","Card"])])
print(d)

OrderedDict([('worst schools', 'you know'), ('best schools', 'cal'), ('cal', 'I mean wow'), ('stanford', "what's French for 'meh'?"), ('Famous Trumps', ['Donald', 'Card'])])


In [18]:
## unlike with a dict, the ordering of each pair in the
## iteration is gauranteed across platforms. Hurray!
for k,v in d.items():
    print(k, "=", v)

worst schools = you know
best schools = cal
cal = I mean wow
stanford = what's French for 'meh'?
Famous Trumps = ['Donald', 'Card']


## namedtuple

"assign meaning to each position in a tuple and allow for more readable, *self-documenting code*."

https://docs.python.org/3/library/collections.html#collections.namedtuple


In [19]:
from collections import namedtuple

In [20]:
Candidate = namedtuple('Candidate', 
                       ['office', 'name', 'party', 'tax_return'])

In [21]:
Candidate._fields

('office', 'name', 'party', 'tax_return')

In [23]:
Candidate.office

In [24]:
Candidate.tax_return.__doc__ = \
   'has the candidate released their tax returns?'

In [25]:
h=Candidate("president","Hillary","Dem",True)
d=Candidate("president","Donald","GOP",False)
k=Candidate("senate","Kamala","Dem",True)

In [26]:
k.party

'Dem'

In [27]:
h._asdict()

OrderedDict([('office', 'president'),
             ('name', 'Hillary'),
             ('party', 'Dem'),
             ('tax_return', True)])

In [28]:
h._asdict().keys()

odict_keys(['office', 'name', 'party', 'tax_return'])

In [ ]:
for cand in [h,d,k]:
    print("{0} ({1}): shown tax return? {2} " \
          .format(cand.name,cand.party,cand.tax_return))

In [29]:
print(Candidate._source)

from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class Candidate(tuple):
    'Candidate(office, name, party, tax_return)'

    __slots__ = ()

    _fields = ('office', 'name', 'party', 'tax_return')

    def __new__(_cls, office, name, party, tax_return):
        'Create new instance of Candidate(office, name, party, tax_return)'
        return _tuple.__new__(_cls, (office, name, party, tax_return))

    @classmethod
    def _make(cls, iterable, new=tuple.__new__, len=len):
        'Make a new Candidate object from a sequence or iterable'
        result = new(cls, iterable)
        if len(result) != 4:
            raise TypeError('Expected 4 arguments, got %d' % len(result))
        return result

    def _replace(_self, **kwds):
        'Return a new Candidate object replacing specified fields with new values'
        result = _self._make(map(kwds.pop, ('office', 'name', 'party', 'tax_return

 &nbsp;

# Making Iterables #

<center>Python can loop over many different types</center>

```python
    >>> for element in [1, 2, 3]:
        print(element, end=" ")
    1 2 3
    >>> for element in (1, 2, 3):
        print(element, end=" ")
    1 2 3
    >>> for key in {'one':1, 'two':2}:
        print(key, end=" ")
    one two
    >>> for char in "123":
        print(char, end=" ")
    1 2 3
    >>> for a in {4,1,3,4,2}:
        print(a, end=" ")
    1 2 3 4
    >>> print({4,1,3,4,2,"a",0j})
        set([0j, 'a', 2, 3, 4, 1])
```

 &nbsp;

# Making Iterables #
<center> Each of those above types have built-in methods.
So do, even, `file` objects: </center>

In [30]:
for l in open("password.file","r"):
    print(l,)

# here's some passwords I cracked

guido  Monty

cleese Python



In [32]:
for x in ["dog","cat","cheezeberger"]:
    print(x, end=" ")

dog cat cheezeberger 

In [33]:
# this is what is actually getting call by the `for` call
# for x in a: ...
a = {"cal": "wow", "stanford": "meh"}
b = iter(a)

In [34]:
b

In [35]:
next(b)

'cal'

In [36]:
next(b)

'stanford'

In [37]:
next(b)

StopIteration: 

In [38]:
b.__next__?

<div class="alert alert-success">
note the consistency with `str()` and `__str__`, `len()` and `__len__`, ...
</div>

 &nbsp;

# Making Iterables #

<p class="gap05"</p>

We can make classes that know how to iterate, becoming new iterables types. The key is to build to special methods: ``.__iter__()`` and ``.__next__()``

<p class="gap03"</p>

 * `.__iter__()` : return an iterator object (usually just self) 
 
 * `.__next__()` : return the next element in the iterator. raise a `StopIteration` exception if there is nothing left
 
<p class="gap05"</p>

https://docs.python.org/3/library/stdtypes.html#typeiter

In [39]:
%%file myits1.py

""" let's make an iterator """
class Reverse(object):
     "Iterator class for looping over a sequence backwards"
     def __init__(self, data):
        self.data = data
        self.index = len(data)

     def __iter__(self):
        # this is a required of an iterating class
        return self

     def __next__(self):
        # we got to the front of the array
        if self.index == 0:
            raise StopIteration
        
        self.index = self.index - 1
        return self.data[self.index]

Writing myits1.py


In [40]:
%run myits1

In [41]:
r = Reverse("god")
print(r)

In [42]:
for c in r: print(c, end=" ")

d o g 

<img src="https://img.buzzfeed.com/buzzfeed-static/static/2014-11/13/17/enhanced/webdr09/enhanced-9483-1415918730-8.png" width="40%">

In [43]:
next(r)

StopIteration: 

In [44]:
r.index = -1

In [45]:
r = Reverse("amanaplanacanalpanama")
for c in r: print(c, end="")

amanaplanacanalpanama

In [46]:
for c in r: print(c, end="")

 &nbsp;

In [47]:
[x**3 for x in range(10)]

[0, 1, 8, 27, 64, 125, 216, 343, 512, 729]

In [48]:
(x**3 for x in range(10))

<generator object <genexpr> at 0x104c6be08>

# Generators #

<p class="gap03"</p>
Create a ``generator`` expression, something that is iterable:
<p class="gap03"</p>

> e.g., (x**2 for x in range(3))

<p class="gap03"</p>
Like list comprehension [] and set comprehension {}

In [49]:
a = ((x,x**2) for x in range(3))

In [50]:
dict([(x,x**2) for x in range(3)])

{0: 0, 1: 1, 2: 4}

In [51]:
for i in (x**2 for x in range(3)):
    print(i, end=" ")

0 1 4 

In [52]:
sum((x for x in range(11)))

55

What's the difference between ( ... ) and [ ... ]?

 &nbsp;

# Making Generators #

<p class="gap03"</p>
we can also make iterables using generating functions
<p class="gap03"</p>

<div class="alert alert-success">Generators are iterators, but you can only iterate over them once. It's because they do not store all the values in memory, they generate the values on the fly</div>

<font color="red"><b>yield</b></font> inside of a function acts like a "temporary return" but saves the entire state of the local variables for further use

In [54]:
%%file first_its.py
def integers():
    """Infinite sequence of integers."""
    i = 1
    while True:
        yield i
        i = i + 1

def squares():
    for i in integers():
        yield i * i

Writing first_its.py


In [55]:
%run first_its.py

In [57]:
for x in integers():
    if x > 10:
        break
    print(x)

1
2
3
4
5
6
7
8
9
10


In [58]:
%%file myits2.py

def countdown(start,end=0,step=1.0):
     i = start
     while (i >= end) or end == None:
          yield i
          i -= step

Writing myits2.py


when the function stops yielding, `StopIteration` is raised (implicitly)

In [59]:
%run myits2

In [60]:
c = countdown(3)

In [61]:
c

<generator object countdown at 0x104d4e7d8>

In [62]:
next(c)

3

In [63]:
next(c)

2.0

In [64]:
next(c)

1.0

"iterator on list: `next()` returns the next element of the list

iterator generator: `next()` will compute the next element on the fly"

http://stackoverflow.com/questions/231767/the-python-yield-keyword-explained

 &nbsp;

# Example: Fibonacci sequence #

$$
F(n) = \left\{ \begin{array}{rl}
  0 &\mbox{ if $n=0$} \\
  1 &\mbox{ if $n=1$} \\
  F(n -1) + F(n -2) & \mbox{ if $n >1$}
       \end{array} \right.
$$

output: 0, 1, 1, 2, 3, 5, 8, 13, 21, ...

In [65]:
%%file myits3.py

def fib():
    a = 0
    b = 1
    i = 0
    while True:
        yield a
        i += 1
        a, b = b, a + b

Writing myits3.py


In [68]:
%run myits3
a = fib()
for i in range(15): print(next(a), end=" ")

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 

In [69]:
%%file myits4.py

def fib1(start=0,end=None,maxnum=100):
    """
another yield example, allowing the user to start their own fibbinoci sequence at
start (default is 0)
    """
    a = start
    b = start + 1
    n_yielded = 0
    while (n_yielded < maxnum or maxnum is None) and ((end is None) or (abs(a) < abs(end))):
        "abs needed to control against silly user starting with a negative number"
        yield a
        n_yielded += 1
        a, b = b, a + b
    
    # if we got here then we are returning instead of yielding. The countdown is finished
    # we could raise a StopException excception here...this is done for us implicitly

Writing myits4.py


In [70]:
%run myits4.py

In [71]:
for e in fib1(start=-1,end=10000,maxnum=10): print(e, end=" ")

-1 0 -1 -1 -2 -3 -5 -8 -13 -21 

In [72]:
b = fib1(start=1,end=10000,maxnum=2)
next(b)

1

In [73]:
next(b)

2

In [74]:
next(b)

StopIteration: 

 &nbsp;

# Breakout! #

<center>
The infinite series:
1 - 1/3 + 1/5 - 1/7 ...
converges to π/4
</center>
<p></p>

a) write a generator function which progressively makes better and better approximations of π.

b) modify the generator to stop after it reaches within 0.1% of the true value of π.  What value do you get?

c) [optional] "accelerate" convergence by writing a generator that takes your answer in a) as an argument and returns:

$$S_n = S_{n+1} - \frac{(S_{n+1} - S_n)^2}{S_{n-1} - 2 S_n + S_{n+1}}$$

<center>
<img src="http://imgon.net/di-JMPP.gif">
</center>

In [77]:
#!/usr/bin/env python
"""
Breakout answer from the Python seminar class (week #1)
  - get's us used to using yield
created by Josh Bloom at UC Berkeley, 2010, 2012, 2016 (ucbpythonclass+seminar@gmail.com)
"""
import math

def pi_series(stop_when_close=False,close=0.00001):
    """ generate the series 4*(1 - 1/3 + 1/5 - 1/7 ...), which should be pi.
      We have Libnitz to thank for this
    """
    thesum = 0.0
    i = 1.0 ; thesign = 1.0
    while ((not stop_when_close) or (abs(4.0*thesum - math.pi) > close*math.pi)):
        thesum += thesign/i
        yield 4.0*thesum
        i += 2.0 
        thesign *= -1.0  ## change the sign in front of that term
    
    # notice that you don't need a StopIteration call

def first_n(g, n):
    for i in range(n):
        yield next(g)

def accel(series):
    """ accelerate the series convergence.... we have Euler to thank for this"""
    s0 = next(series) # Sn-1
    s1 = next(series) # Sn
    s2 = next(series) # Sn+1
    while True:
        yield s2 - ((s2 - s1)**2)/(s0 - 2.0*s1 + s2)
        s0, s1, s2 = s1, s2, next(series)

how_close = 0.001  # 0.1%
firstbunch = \
   list(first_n(pi_series(stop_when_close=True,close=how_close),\
                 1000))

print("{0:.7f} ... {2}% stops after: {1:d} iterations\n"
      .format(firstbunch[-1],len(firstbunch),how_close*100))

print("Last breakout question:") 
print("*"*30)

b = accel(pi_series())
print("fractional accuracy first 8 in accelerated series:")

for i in range(8): 
    print("{:.4e} ".format(abs(next(b) - math.pi)/math.pi))
    
a = pi_series()
print("\n" + "-"*30)
print("\nfractional accuracy first 8 in un-accelerated series:")
for i in range(8): 
    print("{:.4e} ".format(abs(next(a) - math.pi)/math.pi))

3.1447274 ... 0.1% stops after: 319 iterations

Last breakout question:
******************************
fractional accuracy first 8 in accelerated series:
7.9813e-03 
2.6290e-03 
1.1604e-03 
6.0801e-04 
3.5657e-04 
2.2642e-04 
1.5252e-04 
1.0753e-04 

------------------------------

fractional accuracy first 8 in un-accelerated series:
2.7324e-01 
1.5117e-01 
1.0347e-01 
7.8417e-02 
6.3054e-02 
5.2695e-02 
4.5246e-02 
3.9636e-02 


# Itertools #

In [78]:
import itertools

In [79]:
## chain many iterables together
a = itertools.chain((x**2 for x in range(3)), (x**3 for x in range(3)))

In [80]:
for x in a: 
    print(x, end=" ")

0 1 4 0 1 8 

In [81]:
print(a)
print(hasattr(a,"__next__"))
print(hasattr(a,"__iter__"))

True
True


In [82]:
for x in itertools.combinations(["dog","cat","cheezberger"], 2): 
    print(x, end=" ")

('dog', 'cat') ('dog', 'cheezberger') ('cat', 'cheezberger') 

In [83]:
for x in itertools.permutations(["dog","cat","cheezberger"]): 
    print(x, end= " ")

('dog', 'cat', 'cheezberger') ('dog', 'cheezberger', 'cat') ('cat', 'dog', 'cheezberger') ('cat', 'cheezberger', 'dog') ('cheezberger', 'dog', 'cat') ('cheezberger', 'cat', 'dog') 

# Context Managers #

allow you to build classes that provide a context to what you do:
everything inside of a with statement operates abides by the context you create. You decide how to build up the context and how to tear it down.

e.g., holding a lockfile, running a database transaction

```python
>>> with open("password.file","r") as f:
    print f.readlines() 
["# here's some passwords I cracked","guido  Monty","cleese Python"]
```

`f.close()` got called for us (and would have even under an exception)

http://www.python.org/dev/peps/pep-0343/

# Context Managers #

write `__enter__()` and `__exit__()` methods. These get executed no matter what.

In [84]:
%%file myctx1.py

class MyDecor:
    def __enter__(self):
        print("Entered a wonderful technicolor world. Build it up")
        
    def __exit__(self,*args):
        ## *args hold the exception args if needed
        print("...exiting this wonderful world. Tear it down.")

Writing myctx1.py


In [85]:
%run myctx1.py
a = MyDecor()

In [86]:
with MyDecor():
    print(" Do something!")

Entered a wonderful technicolor world. Build it up
 Do something!
...exiting this wonderful world. Tear it down.


`__enter__()` and `__exit__()` only get called when invoked with the with statement

 &nbsp;

In [87]:
%%file myctx2.py

class MyDecor1:
    
    def __init__(self,expression="None"):
        self.expression = expression
    def __enter__(self):
        print("Entered a wonderful technicolor world. Build it up") 
        return eval(self.expression)
    def __exit__(self,*args):
        print("...exiting this wonderful world. Tear it down.")

Writing myctx2.py


In [88]:
%run myctx2
with MyDecor1("2**3") as x:
    print(x)

Entered a wonderful technicolor world. Build it up
8
...exiting this wonderful world. Tear it down.


In [89]:
with MyDecor1("2") as x:
    print(x/0)

Entered a wonderful technicolor world. Build it up
...exiting this wonderful world. Tear it down.


ZeroDivisionError: division by zero

 &nbsp;

<center>
<img src="files/633514032027949357-Interior-Decorators.jpg" width=80%>
</center>

# Decorators #

special functions/classes that augment the functionality of other functions or classes (called in other languages macros or annotations)

denoted with an @sign, immediately preceding decorator name, e.g. `@require_login` or `@testinput`

In [90]:
%%file myctx3.py

def entryExit(f):
    def new_f():
        print("Entering", f.__name__)
        f()
        print("Exited", f.__name__)
    return new_f

@entryExit
def func1():
    print("inside func1()")

@entryExit
def func2():
    print("inside func2()")

Writing myctx3.py


In [91]:
%run myctx3
func1()

Entering func1
inside func1()
Exited func1


In [92]:
func2()

Entering func2
inside func2()
Exited func2


In [93]:
%%file myctx4.py

def introspect(f):
    def wrapper(*arg,**kwarg):
        print("Function name = %s" % f.__name__)
        print(" docstring = %s" % f.__doc__)
        if len(arg) > 0:
            print("   ... got passed args: %s " % str(arg))
        if len(kwarg.keys()) > 0:
            print("   ... got passed keywords: %s " % str(kwarg))
        return f(*arg,**kwarg)
    return wrapper

Writing myctx4.py


<div class="alert alert-info">
Some advantages to using `functools.wraps`:
https://docs.python.org/3/library/functools.html#functools.wraps
</div>

In [94]:
%run myctx4

In [95]:
@introspect
def myrange(start,stop,step):
    return range(start,stop,step)

myrange(1,10,2)

Function name = myrange
 docstring = None
   ... got passed args: (1, 10, 2) 


range(1, 10, 2)

 &nbsp;

In [96]:
def accepts(*types):
    """ Function decorator. Checks that inputs given to decorated function
      are of the expected type.
  
      Parameters:
      types -- The expected types of the inputs to the decorated function.
               Must specify type for each parameter.
    """
    def decorator(f):
        def newf(*args):
            assert len(args) == len(types)
            argtypes = tuple(map(type, args))
            if argtypes != types:
                a = "in %s "  % f.__name__
                a += "got %s but expected %s" % (argtypes,types)
                raise TypeError(a)
            return f(*args)
        return newf
    return decorator

In [97]:
@introspect
@accepts(int,int,int)
def myrange(start,stop,step): 
    return range(start,stop,step)

In [98]:
myrange(1,10,1)

Function name = newf
 docstring = None
   ... got passed args: (1, 10, 1) 


range(1, 10)

In [99]:
myrange(1.0,10,1)

Function name = newf
 docstring = None
   ... got passed args: (1.0, 10, 1) 


TypeError: in myrange got (<class 'float'>, <class 'int'>, <class 'int'>) but expected (<class 'int'>, <class 'int'>, <class 'int'>)

### function annotations

"Function annotations are completely optional metadata information about the types used by user-defined functions." Discussed since 2006.

Annotations are stored in the ``__annotations__`` attribute as a dictionary.

https://docs.python.org/3/tutorial/controlflow.html#function-annotations

In [100]:
def add_ints(x: int, y: int) -> int:
    return x + y

In [101]:
from functools import wraps
from inspect import getcallargs # dictionary

def enforce(f):
    @wraps(f)
    def wrapper(*args, **kws):
        for var, val in getcallargs(f, *args, **kws).items():
            if var in f.__annotations__:
                if type(val) != f.__annotations__.get(var):
                    raise TypeError("{} is not of type {}"
                                   .format(val,f.__annotations__.get(var)))
        ret_value = f(*args, **kws)
        if f.__annotations__.get("return"):
            if type(ret_value) != f.__annotations__.get("return"):
                raise TypeError("{} is not of type {}"
                                   .format(ret_value,f.__annotations__.get("return")))
        return ret_value
    return wrapper

@enforce
def add_ints(x: int, y: int) -> int:
    return x + y

In [102]:
add_ints(1,2)

3

In [103]:
add_ints(1.0,2)

TypeError: 1.0 is not of type <class 'int'>

 &nbsp;

### A Little Teaser: Decorators in Flask #

```python
def requires_roles(*roles):
    def wrapper(f):
        @wraps(f)
        def wrapped(*args, **kwargs):
            if get_current_user_role() not in roles:
                return error_response()
            return f(*args, **kwargs)
        return wrapped
    return wrapper

@app.route('/user')
@required_roles('admin', 'user')
def user_page(self):
    return "You've got permission to access this page."
```

In [104]:
def bread(func):
    def wrapper(*args):
        print("</''''''\>")
        func(*args)
        print("<\______/>")
    return wrapper

def ingredients(func):
    def wrapper(*args):
        print("#tomatoes#")
        func(*args)
        print("~salad~")
    return wrapper

@bread
@ingredients
def sandwich(food="--spam--"):
    print(food)

In [105]:
sandwich("cow")

</''''''\>
#tomatoes#
cow
~salad~
<\______/>


In [106]:
sandwich("--antelope--")

</''''''\>
#tomatoes#
--antelope--
~salad~
<\______/>


see http://stackoverflow.com/questions/739654/understanding-python-decorators

[This was the (unassigned) homework from the bootcamp](https://github.com/profjsb/python-bootcamp/blob/master/Breakouts/Questions/Homework%20Day%20%232.ipynb); you should be able do implement the solution to this...if not, this course will be a challenge to you.

 &nbsp;

<center>
# Enjoy! #

Help online:
   
   <a href="piazza.com/berkeley/fall2016/ay250class13410/home">piazza.com/berkeley/fall2016/ay250class13410/home</a>

See you next **Friday**! Please check the README of this repo for up-to-date reading assignments.

</center>
*remember to email us if you are “sitting in”...*

(c) 2010, 2013, 2016 Python Seminar UC Berkeley, J. S. Bloom All Rights Reserved